In [2]:
import os 
import pandas as pd
from pandas import json_normalize
import json
import requests

from metadata_functions import process_json, create_sponsors_df

api_key = os.environ["CONGRESS_API_KEY"]

### 1. Combine and save raw json files 

In [31]:
# process and save dfs for each congressional session

sessions = [114] # edit me 
metadata_dfs = {}

for session in sessions:

    input_path = f"../data/congress_api_data_10_21/{session}/metadata"
    output_path = f"../data/processed_metadata/{session}_metadata.csv"

    df = process_json(input_path)
    df.to_csv(output_path, index=False)

    metadata_dfs[session] = df

### 2. Call API for specific metadata subjects and save these files

In [32]:
other_data = ['actions','committees','subjects','titles','cosponsors','relatedBills','summaries']

for session in sessions:

    other_data_dict = {}

    for val in other_data:

        column = f"metadata.{val}.url"
        dfs = []

        for bill_id, url in metadata_dfs[session][["bill_id", column]].itertuples(index=False,name=None):

            try:
                response = requests.get(url+f"&api_key={api_key}")
                response_json = response.json()

                df = json_normalize(response_json.get(val))
                df["bill_id"] = bill_id

                dfs.append(df)

            except Exception as e:
                print(e)
                continue

        other_data_dict[val] = pd.concat(dfs, ignore_index=True)

    for key in other_data_dict.keys():

        output_path = f"../data/processed_metadata/{session}_{key}_metadata.csv"
        other_data_dict[key].to_csv(output_path, index=False)
        print(f"Saving data: {session}_{key}")

unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and 'str'
unsupported operand type(s) for +: 'float' and

### Extract sponsor data into new file

In [4]:
## load other data if previous blocks not run

sessions = [110,111,112,113,114,115]

metadata_dict = {}

for session in sessions:

    metadata = pd.read_csv(f"../data/processed_metadata/{session}_metadata.csv")

    metadata_dict[session] = metadata

In [7]:
# create sponsors df

for session in sessions:

    sponsors_df = create_sponsors_df(metadata_dict[session])

    sponsors_df.to_csv(f"../data/processed_metadata/{session}_sponsors_metadata.csv", index=False)